GloVe is a database with a vector of 2.2 million of words
-----------
link [Stanford][1]

Now what is this script doing ? See what a simple script, the beauty is always when its simple.. ;-)

 1. Import the GloVe, 10 rows only, they have 100, 200, 300 vectorrows for the people with more powerful computers, the complete glove is 2Gig... I presume with the 'transfer learning' the aim of this contest is that you build up your own word database like this GloVe. How would you construct that database ? Simple: tfidf all the words, and vectorise with SVD, taking the Vt... I have done hit some notebooks before see [link][2]. So actually i replace the Vt_ with GloVe. Now to be honest i touch the limits of these servers here, they hang more than doing that job... I can't import that database here, i am working on my probably obsolete 4years old singlecored 32bit computer, so this game is also a matter of speed.
  
 2. Tfidf the title and content with the vocabulary of the Glove. top 400.000 words, gives you a very sparse matrix...

 3. Find the good keywords.. What most people are doing with tfidf is simply selecting the most bizar words. Thats not bad, but thats not always the most relevant words in the context. You could use the ranking already of GloVe to find the same thing.
 Now to do the 'transferlearning' IMHO you need to calculate the similarity between all words in the text. Using that Vt_ information or the Glove vector information. What is happening is that the 'co-grouped' words can be selected.

That is my first 10 results...

 - 1  spin particles subatomic 
 - 2  how needed theory prove explanation
   plausible simplest 
 - 3  group groups 
 - 7  
 - 9  principle integral 
 - 13  
 - 15  string disprove 
 - 17  blue color sky 
 - 19  higher energy aim calculated  particles 
 - 21  carlo monte

  
  


  [1]: https://nlp.stanford.edu/projects/glove/
  [2]: https://www.kaggle.com/plarmuseau/transfer-learning-on-stack-exchange-tags/theoretical-transfer-learning-script

In [ ]:
import pandas as pd
# 1  Import glove 10 rows only , i lack memory and speed guys...
Vt_=pd.read_table("../input/glove.6B.50d.txt",usecols=[0,1,2,3,4,5,6,7,8,9,10,11],header=None, delim_whitespace=1,quoting=3)
Vt_.columns=['word',1,2,3,4,5,6,7,8,9,10,11]

#import test database
test = pd.read_csv("../input/testw.csv")
test['category'] = 'physics'

# 2  Tfidf vectorize with the glove.
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf3_vectorizer = TfidfVectorizer(vocabulary=Vt_['word'])
D_ = tfidf3_vectorizer.fit_transform(test['title']+'. '+test['content'])
# 82000*400k woorden 574000 words classified compressed in sparse matrix 300mb
from scipy.sparse import csr_matrix
D_=csr_matrix(D_)
Vword=Vt_['word']
Vt_=Vt_.drop('word', axis=1)

# 3 Find the most relevant words...
for xi in range (0,10):
    idnr=test.ix[xi].id
    Dtemp=pd.DataFrame(D_[xi,:].todense())
    Dtemp=Dtemp.append(Vword.T)
    Dtemp=Dtemp.append(Vt_.T)
    Dtemp=Dtemp.T
    Dtemp.columns=['rf','word','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11']
    Dtemp.sort_values(by='rf')
    Dbis=Dtemp[Dtemp['rf']>0]
    maxrf=Dbis.max(axis=0).rf/2.2
    Dbis=Dtemp[Dtemp['rf']>maxrf]
    Dbis=Dbis.drop(['rf','word'],axis=1)
    Dcorr=Dbis.dot(Dbis.T)/(abs(Dbis).dot(abs(Dbis.T)))
    lengte=len(Dtemp[Dtemp['rf']>0])
    corrm=Dcorr[0:lengte]
    corrm=corrm.fillna(value=0)
    superwoord=corrm[corrm>0.85].sum()
    prnttmp=''
    for wn in superwoord[superwoord>1].index:
        prnttmp+=' '+Dtemp.ix[wn].word
    print(idnr,prnttmp)